In [1]:
LLM_MODEL = 'llama3.2:1b'
EMBED_MODEL = 'BAAI/bge-large-en-v1.5'
RERANK_MODEL = 'BAAI/bge-reranker-base'
INPUT_DIR_PATH = './paul_graham'
COLLECTION_NAME = "document_chat"
EVAL_GENERATOR_MODEL = 'phi3:3.8b'
EVAL_EMBED_MODEL = 'nomic-embed-text'
EVAL_CRITIC_MODEL = 'llama3.2:1b'

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model=LLM_MODEL, temperature=0, request_timeout=120.0)

In [4]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
embed_model = FastEmbedEmbedding(model_name=EMBED_MODEL)

In [5]:
from llama_index.core.postprocessor import SentenceTransformerRerank
reranker = SentenceTransformerRerank(model=RERANK_MODEL, top_n=2)

In [6]:
from qdrant_client import QdrantClient
db_client = QdrantClient(host='localhost', port=6333)

In [7]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
from llama_index.core import SimpleDirectoryReader
loader = SimpleDirectoryReader(
    input_dir= INPUT_DIR_PATH,
    required_exts=['.txt'],
    recursive=True)
docs = loader.load_data()

In [9]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(client= db_client, collection_name= COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents=docs, storage_context=storage_context)

In [10]:
from llama_index.core import PromptTemplate

template = """
    Context information is below.
    -------------------------
    {context_str}
    -------------------------
    Given the context information above, i want you to think
    step by step to answer the query in a crisp manner. Incase
    you don't know the answer, say 'I don't know'.

    Query: {query_str}

    Answer:

"""

qa_template = PromptTemplate(template)

query_engine = index.as_query_engine(similarity_top = 4, node_postprocessors=[reranker])
query_engine.update_prompts(
    {'response_synthesizer:text_qa_template': qa_template}
)
query = """How did the structure of funding startups in batches contribute 
        to the growth and success of the Y Combinator program and the
        startups involved?"""
response = query_engine.query(query)


In [11]:
from IPython.display import display, Markdown
display(Markdown(str(response)))

To address your query, let's break down the steps that contributed to the growth and success of the Y Combinator (YC) program and the startups involved:

1. **Batch Funding**: The founders of YC started funding startups in batches, which allowed them to do things for a large number of startups at once but also provided a more convenient way for founders to receive funding.

2. **Convenience and Scalability**: By doing funding in batches, YC could scale its operations efficiently, allowing it to handle a larger number of startups simultaneously. This facilitated the process of finding suitable startups and providing them with necessary resources.

3. **Isolation and Community Building**: The batch funding model helped create an environment where founders felt more connected to each other. They shared experiences, advice, and support, which fostered a sense of community among the YC alumni. This community became crucial for helping one another in their entrepreneurial journeys.

4. **Founders' Network**: As the number of startups grew, so did the size of the network within YC. The batch funding model enabled founders to connect with each other and gain access to resources, mentorship, and potential investors. This network played a significant role in helping founders navigate the challenges of starting and growing their businesses.

5. **Customer Acquisition**: By providing initial customers to many startups through their batch funding, YC helped accelerate the growth of its alumni companies. This not only benefited the founders but also contributed to the overall success of the Y Combinator program.

6. **Mentorship and Support**: The batch funding model allowed for a more efficient allocation of resources within YC. Founders could receive guidance from experienced mentors, which helped them overcome obstacles and achieve their goals.

7. **Feedback Loop**: As founders received funding from multiple batches, they were able to test and refine their ideas through experimentation with different approaches. This feedback loop enabled the Y Combinator program to continuously improve its selection process and provide better support for its entrepreneurs.

In summary, the structure of funding startups in batches was a key factor in the growth and success of the Y Combinator program and the startups involved. It facilitated scalability, community building, customer acquisition, mentorship, and a feedback loop that helped entrepreneurs refine their ideas and achieve success.

# EVALUATION / TESTING USING Raagas

In [12]:
# for ollama
DEFAULT_REQUEST_TIMEOUT = 120.0

In [13]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('./paul_graham/')
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
documents = loader.load_and_split(text_splitter=text_splitter)

In [14]:
documents[0].to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'document', 'Document'],
 'kwargs': {'metadata': {'source': 'paul_graham/how_to_do_great_things.txt'},
  'page_content': 'How to Do Great Work\n\nJuly 2023\n\nIf you collected lists of techniques for doing great work in a lot of different fields, what would the intersection look like? I decided to find out by making it.\n\nPartly my goal was to create a guide that could be used by someone working in any field. But I was also curious about the shape of the intersection. And one thing this exercise shows is that it does have a definite shape; it\'s not just a point labelled "work hard."\n\nThe following recipe assumes you\'re very ambitious.\n\nThe first step is to decide what to work on. The work you choose needs to have three qualities: it has to be something you have a natural aptitude for, that you have a deep interest in, and that offers scope to do great work.\n\nIn practice you don\'t have to worry much about the thi

In [15]:
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

generator_llm = OllamaLLM(model = EVAL_CRITIC_MODEL)
# critic_llm = Ollama(model= EVAL_CRITIC_MODEL)
embedding_model = OllamaEmbeddings(model = EVAL_EMBED_MODEL) 

In [ ]:
from ragas.testset import TestsetGenerator
generator = TestsetGenerator.from_langchain(
    llm= generator_llm,
    embedding_model= embedding_model
)

testset = generator.generate_with_langchain_docs(
    documents=documents,
    testset_size=10,
    raise_exceptions=False
)
   
# distribution = { simple: 0.5, reasoning: 0.25, multi_context: 0.25}


In [16]:
import pandas as pd
test_df = pd.read_csv('./test/test_data_paul_graham.csv').dropna()

In [17]:
test_df.head()

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,How did the shift to publishing on the web cha...,"[""Wow, I thought, there's an audience. If I wr...",The shift to publishing on the web changed the...,simple,[{'source': 'paul_graham/what_i_worked_on.txt'...,True
1,1,"How does criticizing a project as a ""toy"" rese...","[""[9] You can't usually get paid for doing exa...",Criticizing a project as a 'toy' is similar to...,simple,[{'source': 'paul_graham/how_to_do_great_thing...,True
2,2,How did the structure of funding startups in b...,['The deal for startups was based on a combina...,Funding startups in batches allowed for conven...,simple,[{'source': 'paul_graham/what_i_worked_on.txt'...,True
3,3,How can exploring different topics help in gen...,"[""Talking or writing about the things you're i...",Exploring different topics can help in generat...,simple,[{'source': 'paul_graham/how_to_do_great_thing...,True
4,4,How does focusing consistently on something yo...,"[""The way to beat it is to stop occasionally a...",Great work happens by focusing consistently on...,simple,[{'source': 'paul_graham/how_to_do_great_thing...,True


In [18]:
def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }

In [19]:
from datasets import Dataset
from tqdm.auto import tqdm

test_questions = test_df['question'].values

responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]
dataset_dict = {
    "question": test_questions, 
    "answer": [response["answer"] for response in responses],
    "contexts": [response["contexts"] for response in responses],
    "ground_truth": test_df["ground_truth"].values.tolist(),
    
}
ragas_eval_dataaset = Dataset.from_dict(dataset_dict)

  0%|          | 0/47 [00:00<?, ?it/s]

In [20]:
ragas_eval_dataaset[0]

{'question': 'How did the shift to publishing on the web change the availability of an audience for writers?',
 'answer': "Based on the context information provided, it appears that the shift to publishing online changed the availability of an audience for writers in several ways:\n\n1. **Increased accessibility**: With the advent of the internet, more people had access to writing and publishing opportunities, making it easier for writers to share their work with a wider audience.\n\n2. **Democratization of publishing**: The web enabled writers to self-publish their work without needing traditional gatekeepers like publishers or agents. This democratization of publishing allowed more voices to be heard and gave writers greater control over the dissemination of their content.\n\n3. **Shift from print to online presence**: As writing became more accessible, the need for a physical print platform (like newspapers) decreased, allowing writers to focus on creating digital content that could

In [21]:
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import OllamaLLM

from ragas import evaluate, RunConfig
from ragas.metrics import (faithfulness, answer_correctness, 
    context_recall, context_precision)

In [22]:
metrics = [faithfulness, answer_correctness, context_recall, context_precision]
run_config = RunConfig(timeout=120)


critic_llm = OllamaLLM(model = EVAL_CRITIC_MODEL, format='json')
embed_llm = OllamaEmbeddings(model = EVAL_EMBED_MODEL)

evaluation_result = evaluate(
    llm= critic_llm,
    embeddings= embed_llm, 
    dataset= ragas_eval_dataaset,
    metrics= metrics,
    run_config= run_config
)

Evaluating:   0%|          | 0/188 [00:00<?, ?it/s]

Exception raised in Job[5]: TimeoutError()
Exception raised in Job[0]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[24]: TimeoutError()
Exception raised in Job[25]: TimeoutError()
Exception raised in Job[28]: TimeoutError()
Exception raised in Job[29]: TimeoutError()
Exception raised in Job[32]: TimeoutError()
Exception raised in Job[33]: TimeoutError()
Exception raised in Job[36]: TimeoutError()
Exception raised in Job[37]: TimeoutError()
Exception raised in Job[38]: TimeoutError()
Exception raised in Job[39]: TimeoutError()
Exception raised in Job[40]: TimeoutError()
Exception raised in Job[41]: TimeoutEr

In [23]:
eval_scores_df = pd.DataFrame(evaluation_result.scores)

In [24]:
eval_scores_df.head()

,faithfulness,answer_correctness,context_recall,context_precision
0,NaN,NaN,0.5,1.0
1,NaN,NaN,1.0,1.0
2,NaN,NaN,0.8,1.0
3,0.25,NaN,0.5,1.0
4,NaN,NaN,1.0,1.0
